**Todo**
- Set cover problem
- Random order set cover problem
- Adversarial arrival order model

# Introduction 

The minimum set cover (SC) problem is among the most basic and extensively studied problems in
combinatorial optimization. It represents the canonical covering problem, standing in duality to the family of packing problems.

1. **The Core Problem:** Set Cover (SC)The problem is fundamentally about Set Cover. In the standard SC problem, the goal is to find a minimum-cost sub-collection of sets (from the collection of $m$ sets) such that the union of these chosen sets includes all elements in the universe.

2. **The Dynamic Aspect:** Dynamic universe of at most $n$ elements: This is the key difference from the standard Set Cover problem. The universe $U$ (the set of elements that need to be covered) is not static. Over time, elements are inserted into and deleted from $U$. The algorithm must maintain a valid set cover solution continuously as the universe changes, but the size of the universe at any point is bounded by $n$. ork on the SC problem in the dynamic setting
[AAG+19, AS21, BCH17 , BHI15 , BHN19 , BHNW21, BSZ23 , GKKP17, SU23, SUZ24], where the
universe U undergoes updates (while |U| ≤ n) and the family S of m sets is fixed. The algorithms are
separated to the low-frequency regime (f = O(log n)) and the high-frequency regime (f = Ω(log n)).
The objective is to maintain a near-optimal approximate SC efficiently, where the most well-studied
measure of efficiency is the update time— the time required to update the maintained solution per
update step

3. **The Fixed Parameters and Constraints:** The rest of the sentence describes the static constraints of the available sets and their relationship with the elements:Fixed collection of $m$ sets: The available sets ($S_1, S_2, \dots, S_m$) and the elements they contain do not change. Only the universe $U$ (the elements currently requiring coverage) is dynamic.Each element belongs to at most $f$ sets: This parameter, $f$, is known as the frequency or maximum element degree. It limits the structural complexity of the problem instance. It means that for any single element, it can be covered by a maximum of $f$ different sets in the collection. A smaller $f$ usually means the problem is easier to solve.Each set has cost in $[1/C, 1]$: This defines the cost of selecting a set.The costs are positive and normalized.The most expensive set costs $\mathbf{1}$.The cheapest set costs at least $\mathbf{1/C}$ (where $C$ is a constant, likely $>1$).This ensures all sets have a significant, non-zero, and bounded cost.

# Use Cases

The Dynamic Set Cover (SC) problem with a focus on **worst-case recourse** models real-life scenarios where you must **continuously maintain coverage** of a fluctuating demand using a **fixed set of resources**, and the **cost of changing** the deployed resources (the recourse) is critical.

Here are the primary real-life application areas:

---

## 1. Facility Location and Service Provision 🗺️

This is the most direct application, especially when demand fluctuates rapidly.

* **Emergency Service Allocation (Dynamic):**
    * **Universe ($U$):** The set of residential or business areas that currently **require** emergency service coverage (e.g., active crime hotspots, areas with recent spike in 911 calls, or regions with a power outage). This set is dynamic (elements are inserted/deleted).
    * **Sets ($S$):** The fixed potential locations of **ambulances, fire stations, or police cars** with pre-defined coverage zones.
    * **Goal:** Select the minimum-cost set of active stations/units to cover all required areas ($U$).
    * **Recourse:** The cost associated with **re-deploying** a physical unit (e.g., relocating an ambulance from one station to another to improve coverage). Minimizing recourse ensures the service remains stable and avoids unnecessary movement and downtime.

* **Wireless/Cell Tower Coverage:**
    * **Universe ($U$):** Areas with current low/no signal or a spike in data traffic that need coverage.
    * **Sets ($S$):** Fixed potential locations for small cell towers or relay antennas.
    * **Goal:** Activate a minimum-cost subset of cells to cover the current demand.
    * **Recourse:** The cost/disruption of **activating/deactivating** or **re-tuning** a set of cell towers in response to changing population density (e.g., rush hour, a large event).

---

## 2. Infrastructure and Network Management 🌐

* **Network Monitoring and Sensor Placement:**
    * **Universe ($U$):** The set of critical network links or physical infrastructure components that are currently reporting **anomalies or failures** and must be monitored/diagnosed.
    * **Sets ($S$):** The fixed set of available **monitoring probes or sensors**, each capable of covering a specific subset of links/components.
    * **Goal:** Select the minimum-cost subset of probes to cover all failing components.
    * **Recourse:** The cost/time delay of **re-configuring** or **re-programming** which sensors are actively running a diagnostic (i.e., changing the set of active sets).

* **Airline Crew/Resource Scheduling (Dynamic):**
    * **Universe ($U$):** A set of **flights/legs** that require crew assignments due to last-minute delays or cancellations (the "universe" of unassigned work changes dynamically).
    * **Sets ($S$):** The fixed list of **available flight crews**, where each crew can cover a subset of flights.
    * **Goal:** Select a minimum-cost subset of crews to cover all required flights.
    * **Recourse:** The cost of **changing a crew's assigned schedule** on short notice (e.g., overtime, new accommodation costs, or legal penalties).

The concept of **worst-case recourse** is particularly important here because it forces the algorithm to find a solution that not only minimizes the cost of the *current* selection but also ensures that the **cost of transitioning** to the *next* required solution (when the universe changes) is manageable and bounded, preventing expensive, cascading re-deployments.

In [19]:
def greedy_set_cover(universe, subsets):
    """
    Implements the Greedy Approximation Algorithm for the Set Cover problem.

    Args:
        universe (set): The set of all elements that must be covered. #demand
        subsets (list of sets): A list of available sets, where each set is 
                                a collection of elements. #supply

    Returns:
        list: A list of the chosen subsets that form the cover.
    """
    # 1. Initialize
    # The set of elements that still need to be covered.
    elements_to_cover = universe.copy() 
    # The list of sets we choose for our cover.
    chosen_sets = []

    # 2. Main Loop: Continue until the universe is fully covered
    while elements_to_cover:
        # 3. Find the best set for this iteration
        # The 'best_set' is the one that covers the most UNCOVERED elements.
        
        # We use the max() function with a 'key' to find the subset that 
        # maximizes the size of the intersection with 'elements_to_cover'.
        best_set = max(subsets, 
                       key=lambda s: len(s.intersection(elements_to_cover)))
        
        # 4. Update the chosen cover and the remaining universe
        
        # Get the new elements covered by the best set
        newly_covered_elements = best_set.intersection(elements_to_cover)

        # Remove the newly covered elements from the remaining universe
        elements_to_cover -= newly_covered_elements

        # Add the best set to our final cover
        chosen_sets.append(best_set)

        # Note: In a *weighted* SC problem, you'd choose the set that 
        # maximizes (new elements covered) / (set cost).
        
    return chosen_sets

# --- Example Usage ---

# Define the full universe of elements to be covered
U = set(range(1, 11))  # {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

# Define the available subsets
S = [
    {1, 2, 3, 4},       # S1
    {5, 6, 7, 8},       # S2
    {1, 9, 10},         # S3
    {2, 4, 6, 8, 10},   # S4
    {3, 5, 7, 9}        # S5
]

print(f"Universe U: {U}")
print(f"Available Subsets S: {S}")
print("-" * 30)

# Run the greedy algorithm
result_cover = greedy_set_cover(U, S)

# Print the result
print(f"The chosen set cover has {len(result_cover)} sets:")
for i, s in enumerate(result_cover):
    print(f"  Set {i+1}: {s}")

# Verification: Check if all elements are covered
covered_union = set().union(*result_cover)
print(f"\nAll elements covered: {covered_union == U}")

Universe U: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Available Subsets S: [{1, 2, 3, 4}, {8, 5, 6, 7}, {1, 10, 9}, {2, 4, 6, 8, 10}, {9, 3, 5, 7}]
------------------------------
The chosen set cover has 3 sets:
  Set 1: {2, 4, 6, 8, 10}
  Set 2: {9, 3, 5, 7}
  Set 3: {1, 2, 3, 4}

All elements covered: True


In [19]:
def greedy_set_cover(universe, subsets):
    """
    Implements the Greedy Approximation Algorithm for the Set Cover problem.

    Args:
        universe (set): The set of all elements that must be covered. #demand
        subsets (list of sets): A list of available sets, where each set is 
                                a collection of elements. #supply

    Returns:
        list: A list of the chosen subsets that form the cover.
    """
    # 1. Initialize
    # The set of elements that still need to be covered.
    elements_to_cover = universe.copy() 
    # The list of sets we choose for our cover.
    chosen_sets = []

    # 2. Main Loop: Continue until the universe is fully covered
    while elements_to_cover:
        # 3. Find the best set for this iteration
        # The 'best_set' is the one that covers the most UNCOVERED elements.
        
        # We use the max() function with a 'key' to find the subset that 
        # maximizes the size of the intersection with 'elements_to_cover'.
        best_set = max(subsets, 
                       key=lambda s: len(s.intersection(elements_to_cover)))
        
        # 4. Update the chosen cover and the remaining universe
        
        # Get the new elements covered by the best set
        newly_covered_elements = best_set.intersection(elements_to_cover)

        # Remove the newly covered elements from the remaining universe
        elements_to_cover -= newly_covered_elements

        # Add the best set to our final cover
        chosen_sets.append(best_set)

        # Note: In a *weighted* SC problem, you'd choose the set that 
        # maximizes (new elements covered) / (set cost).
        
    return chosen_sets

# --- Example Usage ---

# Define the full universe of elements to be covered
U = set(range(1, 11))  # {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

# Define the available subsets
S = [
    {1, 2, 3, 4},       # S1
    {5, 6, 7, 8},       # S2
    {1, 9, 10},         # S3
    {2, 4, 6, 8, 10},   # S4
    {3, 5, 7, 9}        # S5
]

print(f"Universe U: {U}")
print(f"Available Subsets S: {S}")
print("-" * 30)

# Run the greedy algorithm
result_cover = greedy_set_cover(U, S)

# Print the result
print(f"The chosen set cover has {len(result_cover)} sets:")
for i, s in enumerate(result_cover):
    print(f"  Set {i+1}: {s}")

# Verification: Check if all elements are covered
covered_union = set().union(*result_cover)
print(f"\nAll elements covered: {covered_union == U}")

Universe U: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Available Subsets S: [{1, 2, 3, 4}, {8, 5, 6, 7}, {1, 10, 9}, {2, 4, 6, 8, 10}, {9, 3, 5, 7}]
------------------------------
The chosen set cover has 3 sets:
  Set 1: {2, 4, 6, 8, 10}
  Set 2: {9, 3, 5, 7}
  Set 3: {1, 2, 3, 4}

All elements covered: True


# Set Covering With Cost Weightings

# Dual implementation

In [31]:
def primal_dual_set_cover(universe, subsets, costs):
    """
    Implements the Primal-Dual Approximation Algorithm for Weighted Set Cover.

    Args:
        universe (set): The set of all elements to be covered.
        subsets (list of sets): The available sets.
        costs (dict): Dictionary mapping frozensets to their costs.

    Returns:
        list: A list of the chosen subsets that form the cover.
    """
    # Initialize
    elements_to_cover = universe.copy() 
    chosen_sets = []
    
    # Dual variables (y_e): Price of each element, starts at 0
    dual_y = {e: 0.0 for e in universe}
    
    # Track the dual sum for each set s: sum(y_e for e in s)
    set_dual_sum = {frozenset(s): 0.0 for s in subsets}

    # Store subsets as frozensets for easy lookup
    f_subsets = [frozenset(s) for s in subsets]

    # Main Loop: Continue until the universe is fully covered
    while elements_to_cover:
        
        # 1. Determine the rate of dual increase for uncovered elements
        # Since we raise all uncovered duals uniformly, the rate is 1.0.

        # 2. Find the "cheapest" time (smallest 'delta') until a set becomes tight
        min_delta = float('inf')
        
        # Find the set that will become tight first
        for s in f_subsets:
            cost_s = costs[s]
            current_dual_sum = set_dual_sum[s]
            
            # The remaining cost for set s is cost_s - current_dual_sum
            remaining_cost = cost_s - current_dual_sum
            
            # Count how many *uncovered* elements are in set s.
            # Only uncovered elements are contributing to the dual sum increase rate.
            uncovered_in_s = len(s.intersection(elements_to_cover))
            
            if uncovered_in_s > 0:
                # Delta is the time needed for each uncovered element's price (y_e)
                # to increase before set s becomes tight.
                delta = remaining_cost / uncovered_in_s
                
                if delta < min_delta:
                    min_delta = delta
                    tight_set = s
        
        # 3. Update all dual variables based on the minimum delta
        for e in elements_to_cover:
            dual_y[e] += min_delta
        
        # 4. Update the dual sums for all sets
        for s in f_subsets:
             # Only the duals of UNCOVERED elements contributed to the increase
            uncovered_in_s = s.intersection(elements_to_cover)
            set_dual_sum[s] += min_delta * len(uncovered_in_s)
        
        # 5. Select the tight set
        if tight_set:
            chosen_sets.append(tight_set)
            
            # 6. Freeze and Repeat: Mark newly covered elements as removed from 'elements_to_cover'
            elements_to_cover -= tight_set
            
        else:
            # Should not happen if the subsets cover the universe
            print("Error: No set became tight.")
            break
            
    return chosen_sets

# --- Example Usage (Using the same setup as before) ---

U = set(range(1, 11))
S = [
    {1, 2, 3, 4},       # S1
    {5, 6, 7, 8},       # S2
    {1, 9, 10},         # S3
    {2, 4, 6, 8, 10},   # S4
    {3, 5, 7, 9}        # S5
]

Set_Costs = {
    frozenset({1, 2, 3, 4}): 0.8,
    frozenset({5, 6, 7, 8}): 0.2,
    frozenset({1, 9, 10}): 0.9,
    frozenset({2, 4, 6, 8, 10}): 1.0,
    frozenset({3, 5, 7, 9}): 0.5
}

# Run the Primal-Dual algorithm
pd_result_cover = primal_dual_set_cover(U, S, Set_Costs)

# Calculate final cost
final_cost_pd = sum(Set_Costs[s] for s in pd_result_cover)

print("## ⚖️ Primal-Dual Weighted Set Cover Solution")
print(f"The chosen set cover has {len(pd_result_cover)} sets:")
for i, s in enumerate(pd_result_cover):
    cost_val = Set_Costs[s]
    print(f"  Chosen Set: {set(s)} (Cost: {cost_val})")

print(f"\n**Total Cost of the Cover: {final_cost_pd:.2f}**")

## ⚖️ Primal-Dual Weighted Set Cover Solution
The chosen set cover has 4 sets:
  Chosen Set: {8, 5, 6, 7} (Cost: 0.2)
  Chosen Set: {9, 3, 5, 7} (Cost: 0.5)
  Chosen Set: {1, 2, 3, 4} (Cost: 0.8)
  Chosen Set: {1, 10, 9} (Cost: 0.9)

**Total Cost of the Cover: 2.40**


# Comparing Duality &
Both the PD and greedy algorithms can be interpreted through the lens of linear programming (LP). The PD algorithm explicitly maintains feasible primal and dual solutions, repeatedly raising dual variables for uncovered elements until some set constraint becomes tight,
and that set is added to the primal solution, terminating with an integral f -approximation. The greedy algorithm admits a dual-fitting analysis: it constructs a dual solution by interpreting the incremental choices of the algorithm as assigning “charges” to covered elements. These dual values may initially violate feasibility, but scaling them down uniformly by the harmonic number H(n)
restores feasibility, yielding the familiar (≈ ln n) approximation through weak duality. Despite its simplicity, the SC problem captures the fundamental structure underlying LP formulations of covering problems and serves as a canonical model for studying approximation and LP-relaxation techniques.

# Dynamic Heuristic Setting

Dynamic Greedy Strategy (Heuristic)In the dynamic setting, we must maintain a set cover $\mathcal{C}$ for the current universe $U$. When an event occurs, we update $\mathcal{C}$ to maintain feasibility while minimizing the change (recourse) and the total set cost.1. Element Insertion ($e$ is added to $U$)If the new element $e$ is already covered by an existing set in $\mathcal{C}$, no action is needed. If $e$ is uncovered, we need to add a set to $\mathcal{C}$.Greedy Choice for Insertion: Select the set $s^* \in \mathcal{S} \setminus \mathcal{C}$ that covers $e$ and offers the best cost-effectiveness for covering $e$ and other currently uncovered elements.$$\text{Choose } s^* = \arg \min_{s: e \in s} \frac{\text{cost}(s)}{\text{New Uncovered Elements Covered by } s}$$Recourse Consideration: Since we only add a set, the recourse is simply the cost of the new set, $\text{cost}(s^*)$. We choose to add a set rather than replacing existing ones to minimize recourse in this step.2. Element Deletion ($e$ is removed from $U$)When an element $e$ is removed from $U$, it may become possible to delete one or more sets from the cover $\mathcal{C}$ if they are no longer necessary.Greedy Choice for Deletion:Check if any set $s \in \mathcal{C}$ is now only covering elements that are redundantly covered (i.e., every element covered by $s$ is also covered by another set in $\mathcal{C} \setminus \{s\}$).If multiple sets can be removed, prioritize removing the most expensive set that results in the largest cost saving.$$\text{Choose } s^{**} = \arg \max_{s \in \mathcal{C}} \left\{ \text{cost}(s) \mid \mathcal{C} \setminus \{s\} \text{ is a valid cover for } U \right\}$$Recourse Consideration: The recourse is the saving, $-\text{cost}(s^{**})$.

In [42]:
# --- 1. Static Weighted Greedy Set Cover (Used for Initialization) ---

def weighted_greedy_set_cover(universe, subsets, costs):
    """
    Implements the Weighted Greedy Approximation Algorithm for the Set Cover problem.
    Used here to generate the initial feasible cover.
    """
    elements_to_cover = universe.copy() 
    chosen_sets = []
    
    # We must use frozensets for set comparison and cost lookup
    f_subsets = [frozenset(s) for s in subsets]
    
    while elements_to_cover:
        best_ratio = float('inf')
        best_set = None

        for s in f_subsets:
            cost = costs[s]
            newly_covered_count = len(s.intersection(elements_to_cover))
            
            if newly_covered_count > 0:
                current_ratio = cost / newly_covered_count
                if current_ratio < best_ratio:
                    best_ratio = current_ratio
                    best_set = s
        
        if best_set:
            elements_to_cover -= best_set.intersection(elements_to_cover)
            chosen_sets.append(best_set)
        else:
            # Should not happen if the subsets cover the universe
            break
            
    total_cost = sum(costs[s] for s in chosen_sets)
    return chosen_sets, total_cost

# --- 2. Dynamic Helper Functions ---

def get_coverage_count(universe, current_cover):
    """Returns a dictionary of how many times each element is currently covered."""
    coverage = {e: 0 for e in universe}
    for s in current_cover:
        for e in s:
            if e in coverage:
                coverage[e] += 1
    return coverage

def is_valid_cover(universe, current_cover):
    """Checks if the current cover covers the entire universe."""
    covered_union = set().union(*current_cover)
    return universe.issubset(covered_union)

# --- 3. Dynamic Update Function (Greedy Heuristic for Recourse) ---

def dynamic_greedy_set_cover_update(universe, all_subsets, costs, current_cover, event, element):
    """
    Handles a single dynamic event (element insertion or deletion) using 
    a simple greedy heuristic for recourse.
    """
    
    # 1. Handle Universe Update
    if event == 'insert':
        universe.add(element)
    elif event == 'delete':
        if element in universe:
            universe.remove(element)
        
    # 2. Check Coverage and Recalculate based on Event

    if event == 'insert':
        is_covered = any(element in s for s in current_cover)
        
        if not is_covered:
            # Greedy Step for INSERTION: Add best cost-effective set
            best_ratio = float('inf')
            best_set = None
            
            # Recalculate which elements are currently UNCOVERED
            covered_elements = set().union(*current_cover)
            elements_to_cover = universe - covered_elements
            
            for s in all_subsets:
                f_s = frozenset(s)
                # Count newly covered elements (must include the new element if it's the target)
                newly_covered_count = len(s.intersection(elements_to_cover | {element}))
                
                # We only consider sets not already in the cover
                if f_s not in current_cover and newly_covered_count > 0:
                    current_ratio = costs[f_s] / newly_covered_count
                    if current_ratio < best_ratio:
                        best_ratio = current_ratio
                        best_set = f_s
            
            if best_set:
                current_cover.append(best_set)
                print(f"  -> INSERTION: Added set {set(best_set)}")
            else:
                print(f"  -> INSERTION: ERROR: Element {element} cannot be covered by any available set.")

    elif event == 'delete':
        # Greedy Step for DELETION: Remove most expensive redundant set
        best_cost_removed = -1
        set_to_remove = None
        
        # Check redundancy of sets currently in the cover
        for s in current_cover:
            # Simple Heuristic: Check if removing 's' still leaves a valid cover for the new universe
            temp_cover = [cs for cs in current_cover if cs != s]
            if is_valid_cover(universe, temp_cover):
                current_cost = costs[s]
                if current_cost > best_cost_removed:
                    best_cost_removed = current_cost
                    set_to_remove = s
        
        if set_to_remove:
            current_cover.remove(set_to_remove)
            print(f"  -> DELETION: Removed set {set(set_to_remove)}")
        else:
            print("  -> DELETION: No set could be safely removed.")

    return universe, current_cover

# --- 4. Setup and Simulation ---

# Original Universe
U_init = set(range(1, 11)) 

# Available Subsets (S1-S5)
S_list = [
    {1, 2, 3, 4},       # S1
    {5, 6, 7, 8},       # S2
    {1, 9, 10},         # S3
    {2, 4, 6, 8, 10},   # S4
    {3, 5, 7, 9}        # S5
]
All_Subsets = S_list

Costs = {
    frozenset({1, 2, 3, 4}): 0.8,
    frozenset({5, 6, 7, 8}): 0.2,
    frozenset({1, 9, 10}): 0.9,
    frozenset({2, 4, 6, 8, 10}): 1.0,
    frozenset({3, 5, 7, 9}): 0.5
}

# Initial Greedy Cover
initial_cover, initial_cost = weighted_greedy_set_cover(U_init.copy(), All_Subsets, Costs)
U_current = U_init.copy()
Current_Cover = initial_cover[:]
Total_Cost = initial_cost

# --- Dynamic Events Simulation ---
print("--- Dynamic Weighted Set Cover Simulation ---")
print(f"### Initial State (Cost: {Total_Cost:.2f})")
print(f"Current Cover: {[set(s) for s in Current_Cover]}")
print("-" * 40)

# Event 1: Element 8 is deleted 
print("EVENT 1: Delete Element 8")

U_current, Current_Cover = dynamic_greedy_set_cover_update(U_current, All_Subsets, Costs, Current_Cover, 'delete', 8)
new_cost = sum(Costs[s] for s in Current_Cover)
recourse = new_cost - Total_Cost
Total_Cost = new_cost
print(f"New Cover: {[set(s) for s in Current_Cover]}, Total Cost: {Total_Cost:.2f}, Recourse: {recourse:.2f}")
print("-" * 40)

# Event 2: Element 11 is inserted (Hypothetical set S6 added for coverage)
print("EVENT 2: Insert Element 11")

# Define a hypothetical new set (S6) and its cost
S6_set = {11, 1, 2}
if frozenset(S6_set) not in Costs:
    Costs[frozenset(S6_set)] = 0.6
    All_Subsets.append(S6_set)

U_current, Current_Cover = dynamic_greedy_set_cover_update(U_current, All_Subsets, Costs, Current_Cover, 'insert', 11)
new_cost = sum(Costs[s] for s in Current_Cover)
recourse = new_cost - Total_Cost
Total_Cost = new_cost
print(f"New Cover: {[set(s) for s in Current_Cover]}, Total Cost: {Total_Cost:.2f}, Recourse: {recourse:.2f}")

--- Dynamic Weighted Set Cover Simulation ---
### Initial State (Cost: 1.90)
Current Cover: [{8, 5, 6, 7}, {1, 2, 3, 4}, {1, 10, 9}]
----------------------------------------
EVENT 1: Delete Element 8
  -> DELETION: No set could be safely removed.
New Cover: [{8, 5, 6, 7}, {1, 2, 3, 4}, {1, 10, 9}], Total Cost: 1.90, Recourse: 0.00
----------------------------------------
EVENT 2: Insert Element 11
  -> INSERTION: Added set {1, 2, 11}
New Cover: [{8, 5, 6, 7}, {1, 2, 3, 4}, {1, 10, 9}, {1, 2, 11}], Total Cost: 2.50, Recourse: 0.60


1. **Initial State Analysis 🚀Initial State (Cost: 1.90)**: The simulation starts with a feasible set cover found by the static Weighted Greedy Algorithm.
Current Cover: [{8, 5, 6, 7}, {1, 2, 3, 4}, {1, 10, 9}]This cover uses three sets:$S_2 = \{5, 6, 7, 8\}$ (Cost 0.2)$S_1 = \{1, 2, 3, 4\}$ (Cost 0.8)$S_3 = \{1, 9, 10\}$ (Cost 0.9)Total Cost: $0.2 + 0.8 + 0.9 = \mathbf{1.90}$

(Note: This differs from the $2.40$ calculated in the previous example's initial run, likely because the algorithm chose a different, cheaper combination of sets $S_1, S_2, S_3$ to cover the universe $\{1, \dots, 10\}$, potentially omitting $S_5$ from the previous run for a net cost saving.)2. Event 1: Element Deletion (No Recourse) 🗑️EVENT 1: Delete Element 8: The element 8 is removed from the universe $U$.Result: -> DELETION: No set could be safely removed.Interpretation: The heuristic checks if any of the three active sets ($\{5, 6, 7, 8\}$, $\{1, 2, 3, 4\}$, or $\{1, 9, 10\}$) are now redundant, meaning all remaining elements they cover are also covered by other chosen sets.The set $S_2 = \{5, 6, 7, 8\}$ now covers $\{5, 6, 7\}$ in the new universe. Since elements $6$ and $7$ might not be covered by the other two sets (and we know from the previous context that $S_5$ was needed to cover 7), removing $S_2$ would leave $\{6\}$ and $\{7\}$ uncovered.Since the cover remained feasible without modification, the Recourse is 0.00. The total cost remains 1.90.3. Event 2: Element Insertion (Positive Recourse) ➕EVENT 2: Insert Element 11: The new element 11 is added to the universe $U$. Since 11 is not in the current cover, the cover must be updated.Greedy Action: The heuristic searches for the most cost-effective set among all available sets to cover 11 (and any other currently uncovered elements).The new, hypothetical set $S_6 = \{1, 2, 11\}$ with Cost 0.6 (as defined in the setup) is chosen.Result: -> INSERTION: Added set {1, 2, 11}New Cover: The new set $\{1, 2, 11\}$ is appended to the existing three sets.Total Cost: The new total cost is $1.90 + 0.60 = \mathbf{2.50}$.Recourse: 0.60: The recourse cost for this event is simply the cost of the single set that was added ($\text{cost}(S_6) = 0.60$).Conclusion: The dynamic heuristic successfully maintained feasibility in both events. The deletion was handled without cost (minimal recourse), and the insertion incurred a recourse cost equal to the cost of the single new set required.

# [Cite:1] Implementation

In [56]:
import math
from typing import Dict, Set, List, Tuple

# --- Core Data Structures for Set Cover (Re-used) ---

class SetCoverInstance:
    """
    Represents the Set Cover instance (U, S) with costs and element-to-set frequency.
    
    The paper defines:
    - U: dynamic universe of at most n elements
    - S: fixed collection of m sets
    - f: max number of sets containing any element (frequency)
    - cost(s): cost of set s in [1/C, 1]
    """
    def __init__(self, elements: Set[int], sets_data: Dict[str, Tuple[Set[int], float]], f_max: int = None, C: float = 1.0):
        """
        :param elements: The current set of elements U.
        :param sets_data: A dictionary {set_id: (elements_in_set, cost)}.
        :param f_max: Maximum frequency f (max number of sets containing an element).
        :param C: Aspect ratio C (used in approximation and recourse bounds).
        """
        self.U = elements
        self.S = sets_data
        self.n = len(elements)
        self.m = len(sets_data)
        self.C = C
        self.f_max = f_max if f_max is not None else self._calculate_f_max()

    def _calculate_f_max(self) -> int:
        """Calculate the max frequency f for initialization."""
        element_counts = {}
        for set_data, (elements, _) in self.S.items():
            for e in elements:
                element_counts[e] = element_counts.get(e, 0) + 1
        return max(element_counts.values()) if element_counts else 0

    def get_set_cost(self, set_id: str) -> float:
        return self.S[set_id][1]
    
    def get_set_elements(self, set_id: str) -> Set[int]:
        return self.S[set_id][0]


# --- Static Greedy Set Cover Algorithm (Re-used) ---

def static_greedy_set_cover(instance: SetCoverInstance) -> Tuple[Set[str], float, Dict[int, float]]:
    """
    Implements the classic log n-approximation greedy algorithm.
    This algorithm's solution is proven to be 'robust' in the paper (Observation 5).
    """
    U_uncovered = instance.U.copy()
    S_chosen = set()
    total_cost = 0.0
    
    # Store the marginal cost per element (dual variables in the analysis)
    element_weights: Dict[int, float] = {}

    print(f"Starting Greedy with {len(U_uncovered)} uncovered elements...")

    while U_uncovered:
        best_set = None
        min_cost_effectiveness = float('inf')
        
        # Determine the set with the best cost-effectiveness
        for set_id, (elements, cost) in instance.S.items():
            if set_id in S_chosen:
                continue

            # Elements covered by this set that are still uncovered
            newly_covered = elements.intersection(U_uncovered)
            num_newly_covered = len(newly_covered)

            if num_newly_covered > 0:
                cost_effectiveness = cost / num_newly_covered
                
                if cost_effectiveness < min_cost_effectiveness:
                    min_cost_effectiveness = cost_effectiveness
                    best_set = set_id
        
        if best_set is None:
            break

        # Select the best set and update the cover
        S_chosen.add(best_set)
        total_cost += instance.get_set_cost(best_set)
        
        newly_covered_by_best_set = instance.get_set_elements(best_set).intersection(U_uncovered)
        marginal_cost = min_cost_effectiveness
        for e in newly_covered_by_best_set:
            element_weights[e] = marginal_cost 
        
        U_uncovered.difference_update(newly_covered_by_best_set)
        print(f"  --> Selected {best_set} (Cost: {instance.get_set_cost(best_set):.2f}, Covers: {len(newly_covered_by_best_set)}) | Uncovered remaining: {len(U_uncovered)}")


    return S_chosen, total_cost, element_weights


# --- Conceptual Dynamic Set Cover (Re-used) ---

class DynamicSetCover:
    """
    Conceptual structure for a dynamic set cover algorithm, designed to maintain
    an approximate SC under element updates, following the paper's framework.
    """
    def __init__(self, sets_data: Dict[str, Tuple[Set[int], float]], C: float, f: int, epsilon: float):
        self.S = sets_data
        self.C = C
        self.f = f
        self.epsilon = epsilon
        self.U_alive: Set[int] = set()
        self.current_cover: Set[str] = set()
        self.recourse_counter = 0

    def update(self, update_type: str, element: int):
        """Handles an update (Element Insertion or Deletion)."""
        initial_cover = self.current_cover.copy()
        
        if update_type == "INSERT":
            self._handle_insertion(element)
        elif update_type == "DELETE":
            self._handle_deletion(element)

        # --- Recourse Measurement ---
        sets_added = self.current_cover.difference(initial_cover)
        sets_removed = initial_cover.difference(self.current_cover)
        recourse = len(sets_added) + len(sets_removed)
        self.recourse_counter += recourse
        
        # This printout simulates the complexity analysis in the paper:
        print(f"Update: {update_type} {element}. Recourse: {recourse} change(s). Cover size: {len(self.current_cover)}")


    def _handle_insertion(self, element: int):
        if element not in self.U_alive:
            self.U_alive.add(element)
            
            # Simple check: if element is now uncovered, find a set to cover it
            is_covered = any(element in self.S[s_id][0] for s_id in self.current_cover)
            
            if not is_covered:
                # In the low-recourse paper, this triggers the complex invariant-maintaining logic
                # For simulation, we just select the lowest-cost set covering it.
                best_s_id = None
                min_cost = float('inf')
                for s_id, (elements, cost) in self.S.items():
                    if element in elements and cost < min_cost:
                        min_cost = cost
                        best_s_id = s_id
                        
                if best_s_id:
                    self.current_cover.add(best_s_id)
                    # The full SUZ24 algorithm would update lev, plev, and assignments here.


    def _handle_deletion(self, element: int):
        if element in self.U_alive:
            self.U_alive.remove(element)
            # A deletion makes an element 'dead'. The full algorithm would check if 
            # this deletion now allows for a low-recourse optimization/swap, which 
            # is the core of the paper's contribution.

    def get_cover(self) -> Set[str]:
        return self.current_cover

    def get_approximation_factor(self, n) -> str:
        """Based on Theorem 3 from the paper (for high-frequency)."""
        return f"Approx. Factor: (2 + \u03B5) ln n \u2248 ({2 + self.epsilon:.2f} * ln({n}))"


# ==============================================================================
# --- Example Usage with New Data ---

# Input Data
S_list = [
    {1, 2, 3, 4},       # S1
    {5, 6, 7, 8},       # S2
    {1, 9, 10},         # S3
    {2, 4, 6, 8, 10},   # S4
    {3, 5, 7, 9}        # S5
]
Costs = {
    frozenset({1, 2, 3, 4}): 0.8,
    frozenset({5, 6, 7, 8}): 0.2,
    frozenset({1, 9, 10}): 0.9,
    frozenset({2, 4, 6, 8, 10}): 1.0,
    frozenset({3, 5, 7, 9}): 0.5
}

# 1. Format Data for the Instance Class
INITIAL_ELEMENTS = set.union(*S_list)
C_FACTOR = 1.0  # Assuming unweighted costs for C=1, as costs are in [0, 1]
EPSILON = 0.1

SETS = {}
for i, s in enumerate(S_list):
    set_id = f'S{i+1}'
    # Lookup the cost using the frozenset as the key
    cost = Costs[frozenset(s)] 
    SETS[set_id] = (s, cost)

# 2. Run the Static Greedy Algorithm
static_instance = SetCoverInstance(INITIAL_ELEMENTS, SETS, C=C_FACTOR)
print("="*50)
print("--- 1. Static Greedy Set Cover (Approximation Basis) ---")
print(f"Initial Universe (n): {static_instance.n}")
print(f"Max Frequency (f): {static_instance.f_max}")
print("-" * 40)

S_static, cost_static, weights = static_greedy_set_cover(static_instance)

print("-" * 40)
print(f"Final Sets Chosen: {S_static}")
print(f"Total Cost: {cost_static:.2f}")
print(f"Greedy Approx. Factor (H_n): {math.log(static_instance.n) + 0.577:.2f} (approx. ln n)")
print("="*50 + "\n")


# 3. Run the Conceptual Dynamic Set Cover Simulation
dynamic_sc = DynamicSetCover(SETS, C=C_FACTOR, f=static_instance.f_max, epsilon=EPSILON)

# Initialize the dynamic cover with the greedy solution
dynamic_sc.current_cover = S_static.copy()
dynamic_sc.U_alive = INITIAL_ELEMENTS.copy()
print("--- 2. Conceptual Dynamic Set Cover Simulation ---")
print(dynamic_sc.get_approximation_factor(static_instance.n))
print(f"Worst-Case Recourse Bound (O(\u03B1C)): O({(math.log(static_instance.n) + 0.577) * C_FACTOR:.2f}) based on the paper's Theorem 1.")
print("-" * 40)

# Simulate updates (Element Insertion/Deletion)
print("Simulating Updates:")
dynamic_sc.update("DELETE", 2)
dynamic_sc.update("INSERT", 11) # New element
dynamic_sc.update("DELETE", 5)
dynamic_sc.update("INSERT", 12) # Another new element

print("-" * 40)
print(f"Total Updates Simulated: 4")
print(f"Final Cover Sets: {dynamic_sc.get_cover()}")
print(f"Total Recourse: {dynamic_sc.recourse_counter} (cumulative changes to the cover)")

--- 1. Static Greedy Set Cover (Approximation Basis) ---
Initial Universe (n): 10
Max Frequency (f): 2
----------------------------------------
Starting Greedy with 10 uncovered elements...
  --> Selected S2 (Cost: 0.20, Covers: 4) | Uncovered remaining: 6
  --> Selected S1 (Cost: 0.80, Covers: 4) | Uncovered remaining: 2
  --> Selected S3 (Cost: 0.90, Covers: 2) | Uncovered remaining: 0
----------------------------------------
Final Sets Chosen: {'S1', 'S2', 'S3'}
Total Cost: 1.90
Greedy Approx. Factor (H_n): 2.88 (approx. ln n)

--- 2. Conceptual Dynamic Set Cover Simulation ---
Approx. Factor: (2 + ε) ln n ≈ (2.10 * ln(10))
Worst-Case Recourse Bound (O(αC)): O(2.88) based on the paper's Theorem 1.
----------------------------------------
Simulating Updates:
Update: DELETE 2. Recourse: 0 change(s). Cover size: 3
Update: INSERT 11. Recourse: 0 change(s). Cover size: 3
Update: DELETE 5. Recourse: 0 change(s). Cover size: 3
Update: INSERT 12. Recourse: 0 change(s). Cover size: 3
-------

## References

  * Solomon, S., & Uzrad, A. (2025). *Dynamic Set Cover with Worst-Case Recourse*. arXiv:2511.07354. [cite\_start][https://arxiv.org/abs/2511.07354](https://www.google.com/search?q=https://arxiv.org/abs/2511.07354) [cite: 1]